In [1067]:
import gspread
import pycountry
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from google.auth import default

from __future__ import print_function
from datetime import datetime
from datetime import timedelta
import os.path
import requests
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

import pandas as pd
import numpy as np
import seaborn as sns
import re
import logging
from math import trunc
import json
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import scipy.stats as stats
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

#### Connect to Google Sheets

In [1068]:
# Set up scopes and load credentials
scopes = ["https://www.googleapis.com/auth/spreadsheets"]
credentials_path = "wfm-automations.json"
creds = service_account.Credentials.from_service_account_file(credentials_path, scopes=scopes)

# creds.refresh(Request()) #manually refresh the credentials

gc = gspread.authorize(creds)

sheet_id = "1cbW6p8rdMhq9XD5WL-QJNO1UBRQRTcKQBoh6g1QfEv8" # Marketing Forecasts 2024/25 copied file
sheet = gc.open_by_key(sheet_id)
worksheet = sheet.worksheet("actual_budget_2025")

worksheet

<Worksheet 'actual_budget_2025' id:733883154>

### Steps

1. Takie CUM and NET MAUs and transpose it to a sheet

2. Extrapolate at least 2 years of data

3. Map it with the Langauge % split


### Step 1 - Extracting numbers from marketing file

In [1069]:
# net maus
cell_range = worksheet.get('A11:O19')
net_mau = pd.DataFrame(cell_range)
net_mau

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Net MAU,Nov 24,Dec 24,Jan 25,Feb 25,Mar 25,Apr 25,May 25,Jun 25,Jul 25,Aug 25,Sep 25,Oct 25,Nov 25,Dec 25
1,Germany,"17,776","20,210","24,733","21,222","24,888","22,424","24,529","24,749","25,586","27,904","33,626","35,281","31,216","26,543"
2,Austria,"1,842","2,451","2,575","2,700","2,962","2,976","3,068","2,864","2,859","2,797","3,363","3,136","2,583","2,233"
3,France,"8,295","7,918","9,308","8,912","11,815","11,610","11,756","10,581","11,781","13,078","15,603","17,330","16,085","11,325"
4,Italy,146,603,-843,-827,-892,-878,-894,"2,141","2,628","4,329","5,637","3,320","2,850","1,639"
5,Spain,"6,643","6,375","8,014","9,009","9,970","10,298","10,932","9,499","10,218","10,506","14,101","16,155","14,648","12,385"
6,Greater Europe,"1,209","1,765","1,070","2,052","2,915","3,735","4,837","5,861","6,760","7,188","10,412","10,583","9,413","5,001"
7,Non-Euro,269,529,574,"1,077",814,487,480,389,356,322,283,361,422,525
8,Total,"36,180","39,852","45,431","44,145","52,472","50,652","54,708","56,084","60,188","66,124","83,025","86,166","77,217","59,651"


In [1070]:
#cum maus
cell_range = worksheet.get('A1:O9')
cum_mau = pd.DataFrame(cell_range)
cum_mau

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Cum. Net MAU,Nov 24,Dec 24,Jan 25,Feb 25,Mar 25,Apr 25,May 25,Jun 25,Jul 25,Aug 25,Sep 25,Oct 25,Nov 25,Dec 25
1,Germany,"1,086,666","1,106,876","1,131,609","1,152,831","1,177,719","1,200,143","1,224,672","1,249,421","1,275,007","1,302,911","1,336,537","1,371,818","1,403,034","1,429,577"
2,Austria,"108,248","110,699","113,274","115,974","118,936","121,912","124,980","127,844","130,703","133,500","136,863","139,999","142,582","144,815"
3,France,"446,451","454,369","463,677","472,589","484,404","496,014","507,770","518,351","530,132","543,210","558,813","576,143","592,228","603,553"
4,Italy,"212,951","213,554","212,711","211,884","210,992","210,114","209,220","211,361","213,989","218,318","223,955","227,275","230,125","231,764"
5,Spain,"298,750","305,125","313,139","322,148","332,118","342,416","353,348","362,847","373,065","383,571","397,672","413,827","428,475","440,860"
6,Greater Europe,"209,482","211,248","212,318","214,370","217,285","221,020","225,857","231,718","238,478","245,666","256,078","266,661","276,074","281,075"
7,Non-Euro,"28,821","29,350","29,924","31,001","31,815","32,302","32,782","33,171","33,527","33,849","34,132","34,493","34,915","35,440"
8,Total,"2,391,369","2,431,220","2,476,651","2,520,796","2,573,268","2,623,920","2,678,628","2,734,712","2,794,900","2,861,024","2,944,049","3,030,215","3,107,432","3,167,083"


### Step 2. Transpose data

In [1071]:
# create converter sheet to transpose and extrapoalte data
try:
    converter_worksheet = sheet.add_worksheet(title="Converter", rows=100, cols=20)
except gspread.exceptions.APIError:
    converter_worksheet = sheet.worksheet("Converter")
    converter_worksheet.clear()

cum_mau_values = cum_mau.values.tolist()
converter_worksheet.update("A1", cum_mau_values)

/var/folders/ld/t4l42g9d2kndcwfk5qh21smc0000gp/T/ipykernel_9527/1174368924.py:9: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  converter_worksheet.update("A1", cum_mau_values)


{'spreadsheetId': '1cbW6p8rdMhq9XD5WL-QJNO1UBRQRTcKQBoh6g1QfEv8',
 'updatedRange': 'Converter!A1:O9',
 'updatedRows': 9,
 'updatedColumns': 15,
 'updatedCells': 135}

In [1072]:
# access the converter sheet data
converter_data = converter_worksheet.get_all_records() 
df_converter = pd.DataFrame(converter_data)
df_converter


,Cum. Net MAU,Nov 24,Dec 24,Jan 25,Feb 25,Mar 25,Apr 25,May 25,Jun 25,Jul 25,Aug 25,Sep 25,Oct 25,Nov 25,Dec 25
0,Germany,1086666,1106876,1131609,1152831,1177719,1200143,1224672,1249421,1275007,1302911,1336537,1371818,1403034,1429577
1,Austria,108248,110699,113274,115974,118936,121912,124980,127844,130703,133500,136863,139999,142582,144815
2,France,446451,454369,463677,472589,484404,496014,507770,518351,530132,543210,558813,576143,592228,603553
3,Italy,212951,213554,212711,211884,210992,210114,209220,211361,213989,218318,223955,227275,230125,231764
4,Spain,298750,305125,313139,322148,332118,342416,353348,362847,373065,383571,397672,413827,428475,440860
5,Greater Europe,209482,211248,212318,214370,217285,221020,225857,231718,238478,245666,256078,266661,276074,281075
6,Non-Euro,28821,29350,29924,31001,31815,32302,32782,33171,33527,33849,34132,34493,34915,35440
7,Total,2391369,2431220,2476651,2520796,2573268,2623920,2678628,2734712,2794900,2861024,2944049,3030215,3107432,3167083


In [1073]:
#rename Cum. Net MAU with country
df_converter = df_converter.rename(columns={"Cum. Net MAU": "Country"})

In [1074]:
df_converter.columns

Index(['Country', ' Nov 24', ' Dec 24', ' Jan 25', ' Feb 25', ' Mar 25',
       ' Apr 25', ' May 25', ' Jun 25', ' Jul 25', ' Aug 25', ' Sep 25',
       ' Oct 25', ' Nov 25', ' Dec 25'],
      dtype='object')

#### Step 3. Extrapolate with Linear Regression

In [1075]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def format_and_forecast_data(df_converter):
    # Create a list of all months (historical + forecast)
    historical_months = df_converter.columns[1:].tolist()  # Skip the Country column
    
    # Generate forecast months through December 2027
    forecast_months = []
    for year in [26, 27]:  # Loop through years 2026 and 2027
        for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:
            forecast_months.append(f'{month} {year}')
    
    all_months = historical_months + forecast_months
    
    # Initialize the result DataFrame with the Country column
    result_df = pd.DataFrame({'Country': df_converter['Country']})
    
    # Prepare data for forecasting
    for country in df_converter['Country']:
        # Get historical data for this country
        historical_values = df_converter[df_converter['Country'] == country].iloc[:, 1:].values.flatten()
        
        # Prepare data for linear regression
        X = np.arange(len(historical_values)).reshape(-1, 1)
        y = historical_values
        
        # Fit linear regression model
        model = LinearRegression()
        model.fit(X, y)
        
        # Generate forecasts for 24 months (2 years)
        future_X = np.arange(len(historical_values), 
                           len(historical_values) + len(forecast_months)).reshape(-1, 1)
        forecasted_values = model.predict(future_X)
        
        # Combine historical and forecasted values
        all_values = np.concatenate([historical_values, forecasted_values])
        
        # Add to result DataFrame
        country_mask = result_df['Country'] == country
        for month, value in zip(all_months, all_values):
            result_df.loc[country_mask, month] = value
    
    return result_df

In [1076]:
# Use function
final_df = format_and_forecast_data(df_converter)
final_df.iloc[:, 1:] = final_df.iloc[:, 1:].round(1)
final_df

,Country,Nov 24,Dec 24,Jan 25,Feb 25,Mar 25,Apr 25,May 25,Jun 25,Jul 25,...,Mar 27,Apr 27,May 27,Jun 27,Jul 27,Aug 27,Sep 27,Oct 27,Nov 27,Dec 27
0,Germany,1086666.0,1106876.0,1131609.0,1152831.0,1177719.0,1200143.0,1224672.0,1249421.0,1275007.0,...,1815206.2,1841664.9,1868123.6,1894582.3,1921041.0,1947499.7,1973958.4,2000417.1,2026875.8,2053334.5
1,Austria,108248.0,110699.0,113274.0,115974.0,118936.0,121912.0,124980.0,127844.0,130703.0,...,188584.2,191474.1,194363.9,197253.8,200143.7,203033.5,205923.4,208813.3,211703.1,214593.0
2,France,446451.0,454369.0,463677.0,472589.0,484404.0,496014.0,507770.0,518351.0,530132.0,...,781434.4,793701.5,805968.5,818235.6,830502.6,842769.6,855036.7,867303.7,879570.7,891837.8
3,Italy,212951.0,213554.0,212711.0,211884.0,210992.0,210114.0,209220.0,211361.0,213989.0,...,249752.8,251275.5,252798.2,254320.9,255843.5,257366.2,258888.9,260411.6,261934.2,263456.9
4,Spain,298750.0,305125.0,313139.0,322148.0,332118.0,342416.0,353348.0,362847.0,373065.0,...,598116.5,609100.8,620085.1,631069.4,642053.7,653038.0,664022.3,675006.6,685990.9,696975.1
5,Greater Europe,209482.0,211248.0,212318.0,214370.0,217285.0,221020.0,225857.0,231718.0,238478.0,...,360275.3,366044.5,371813.6,377582.8,383352.0,389121.2,394890.4,400659.6,406428.7,412197.9
6,Non-Euro,28821.0,29350.0,29924.0,31001.0,31815.0,32302.0,32782.0,33171.0,33527.0,...,43147.1,43640.6,44134.1,44627.6,45121.1,45614.5,46108.0,46601.5,47095.0,47588.5
7,Total,2391369.0,2431220.0,2476651.0,2520796.0,2573268.0,2623920.0,2678628.0,2734712.0,2794900.0,...,4036515.1,4096900.3,4157285.5,4217670.7,4278055.9,4338441.1,4398826.3,4459211.5,4519596.7,4579981.9


In [1077]:
df_converter.columns

Index(['Country', ' Nov 24', ' Dec 24', ' Jan 25', ' Feb 25', ' Mar 25',
       ' Apr 25', ' May 25', ' Jun 25', ' Jul 25', ' Aug 25', ' Sep 25',
       ' Oct 25', ' Nov 25', ' Dec 25'],
      dtype='object')

In [1078]:
# access mapping table in the marketing file

cell_range = worksheet.get('A22:F29')
lang_table = pd.DataFrame(cell_range)
lang_table

,0,1,2,3,4,5
0,TnC Country,de,en,es,fr,it
1,AUT,80.09%,18.12%,1.08%,0.22%,0.48%
2,DEU,64.63%,30.31%,3.35%,0.84%,0.88%
3,ESP,0.81%,18.17%,78.87%,1.30%,0.85%
4,FRA,0.26%,7.91%,1.33%,90.22%,0.28%
5,GrE,1.84%,84.89%,3.68%,8.85%,0.75%
6,ITA,1.22%,8.26%,1.33%,0.23%,88.96%
7,NEuro,22.27%,61.00%,5.37%,8.20%,3.16%


In [1079]:
# set index column
lang_table.columns = lang_table.iloc[0] 
lang_table = lang_table[1:] 
lang_table = lang_table.reset_index(drop=True)
lang_table

,TnC Country,de,en,es,fr,it
0,AUT,80.09%,18.12%,1.08%,0.22%,0.48%
1,DEU,64.63%,30.31%,3.35%,0.84%,0.88%
2,ESP,0.81%,18.17%,78.87%,1.30%,0.85%
3,FRA,0.26%,7.91%,1.33%,90.22%,0.28%
4,GrE,1.84%,84.89%,3.68%,8.85%,0.75%
5,ITA,1.22%,8.26%,1.33%,0.23%,88.96%
6,NEuro,22.27%,61.00%,5.37%,8.20%,3.16%


In [1080]:
final_df = final_df.T

In [1081]:
#setting cum maus.T into a new tab called Finals
if "Country" in final_df.columns:
    final_df = final_df.drop(columns=["Country"])
    
final_df.insert(0, "Country", final_df.index)

final_df.columns = final_df.iloc[0]  # Set the first row as column headers
final_df = final_df[1:].reset_index(drop=True)  # Remove the first row from data
final_df_values = [final_df.columns.tolist()] + final_df.values.tolist()

try:
    final_worksheet = sheet.add_worksheet(title="Final", rows=100, cols=20)
except gspread.exceptions.APIError:
    final_worksheet = sheet.worksheet("Final")
    final_worksheet.clear()

final_worksheet.update("A1", final_df_values)


/var/folders/ld/t4l42g9d2kndcwfk5qh21smc0000gp/T/ipykernel_9527/210858566.py:17: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  final_worksheet.update("A1", final_df_values)


{'spreadsheetId': '1cbW6p8rdMhq9XD5WL-QJNO1UBRQRTcKQBoh6g1QfEv8',
 'updatedRange': 'Final!A1:I39',
 'updatedRows': 39,
 'updatedColumns': 9,
 'updatedCells': 351}

In [ ]:
# Next steps to be added in a later stage